In [2]:
from pyspark.sql.functions import concat_ws, col
from pyspark import SparkContext, SparkConf, StorageLevel
from pyspark.sql import SQLContext
import settings, os, re, itertools, json
from pyspark.ml.feature import Tokenizer, RegexTokenizer
products = sqlContext.read.json('../data_input/products.txt')



In [3]:
listings = sqlContext.read.json('../data_input/listings.txt')

In [2]:
d = products.withColumnRenamed('announced-date', 'id')\
    .select(col('id'), concat_ws(' ', products.product_name, products.family, products.model).alias('name'), products.manufacturer)\
    .map(lambda row: (row.id, {'name': row.name, 'manufacturer': row.manufacturer}))\
    .collectAsMap()
print d

{u'2009-07-13T20:00:00.000-04:00': {'name': u'Samsung_SL720 SL720', 'manufacturer': u'Samsung'}, u'2011-02-06T19:00:00.000-05:00': {'name': u'Canon-T3 Rebel T3', 'manufacturer': u'Canon'}, u'1999-06-29T20:00:00.000-04:00': {'name': u'Sony_Cyber-shot_DSC-D770 Cyber-shot DSC-D770', 'manufacturer': u'Sony'}, u'2000-04-25T20:00:00.000-04:00': {'name': u'Sony_Cyber-shot_DSC-F55V Cyber-shot DSC-F55V', 'manufacturer': u'Sony'}, u'2010-05-02T20:00:00.000-04:00': {'name': u'Samsung_WB5500 WB5500', 'manufacturer': u'Samsung'}, u'2009-01-15T19:00:00.000-05:00': {'name': u'Panasonic_Lumix_DMC-FS15 Lumix DMC-FS15', 'manufacturer': u'Panasonic'}, u'2000-05-14T20:00:00.000-04:00': {'name': u'Kodak_DCS620x DCS620x', 'manufacturer': u'Kodak'}, u'2010-07-16T20:00:00.000-04:00': {'name': u'Panasonic_Lumix_DMC-FZ100 Lumix DMC-FZ100', 'manufacturer': u'Panasonic'}, u'2000-06-01T20:00:00.000-04:00': {'name': u'Fujifilm_FinePix_40i FinePix 40i', 'manufacturer': u'Fujifilm'}, u'2009-06-11T20:00:00.000-04:00':

In [6]:
products_dict = d
listings.show()

+--------+--------------------+------+--------------------+
|currency|        manufacturer| price|               title|
+--------+--------------------+------+--------------------+
|     CAD|Neewer Electronic...| 35.99|LED Flash Macro R...|
|     CAD|        Canon Canada|199.96|Canon PowerShot S...|
|     CAD|        Canon Canada|209.00|Canon PowerShot S...|
|     CAD|        Canon Canada|306.24|Canon PowerShot D...|
|     CAD|        Canon Canada|420.33|Canon PowerShot D...|
|     CAD|             Samsung| 13.99|Genuine Samsung E...|
|     CAD|        Canon Canada|129.99|Canon PowerShot A...|
|     CAD|        Canon Canada| 88.00|Canon PowerShot A...|
|     CAD|        Canon Canada|129.92|Canon PowerShot A...|
|     CAD|        Canon Canada|259.99|Canon PowerShot E...|
|     CAD|        Canon Canada|289.00|Canon PowerShot E...|
|     CAD|        Canon Canada|313.33|Canon PowerShot S...|
|     CAD|               Kodak|299.00|Kodak EasyShare Z...|
|     CAD|               Kodak|299.95|Ko

In [7]:
listings_dict = listings.select(concat_ws(' ', listings.currency, listings.price).alias('id'), listings.manufacturer, listings.title)\
                         .map(lambda row: (row.id, {'name': row.title, 'manufacturer': row.manufacturer}))\
                         .collectAsMap()

In [ ]:
tokenizer = Tokenizer(inputCol="title", outputCol="words")
listings = tokenizer.transform(listings)


In [6]:
listings.show()

+--------+--------------------+------+--------------------+--------------------+
|currency|        manufacturer| price|               title|               words|
+--------+--------------------+------+--------------------+--------------------+
|     CAD|Neewer Electronic...| 35.99|LED Flash Macro R...|[led, flash, macr...|
|     CAD|        Canon Canada|199.96|Canon PowerShot S...|[canon, powershot...|
|     CAD|        Canon Canada|209.00|Canon PowerShot S...|[canon, powershot...|
|     CAD|        Canon Canada|306.24|Canon PowerShot D...|[canon, powershot...|
|     CAD|        Canon Canada|420.33|Canon PowerShot D...|[canon, powershot...|
|     CAD|             Samsung| 13.99|Genuine Samsung E...|[genuine, samsung...|
|     CAD|        Canon Canada|129.99|Canon PowerShot A...|[canon, powershot...|
|     CAD|        Canon Canada| 88.00|Canon PowerShot A...|[canon, powershot...|
|     CAD|        Canon Canada|129.92|Canon PowerShot A...|[canon, powershot...|
|     CAD|        Canon Cana

In [2]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer

sentenceDataFrame = sqlContext.createDataFrame([
    (0, "Hi I heard about Spark"),
    (1, "Ricoh - UnitÃ© P10 28-300 mm pour appareil photo GXR"),
    (2, "Logistic,regression,models,are,neat")
], ["label", "sentence"])
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
wordsDataFrame = tokenizer.transform(sentenceDataFrame)
for words_label in wordsDataFrame.select("words", "label").take(3):
    print(words_label)
regexTokenizer = RegexTokenizer(inputCol="sentence", outputCol="words", pattern="\\W")

Row(words=[u'hi', u'i', u'heard', u'about', u'spark'], label=0)
Row(words=[u'ricoh', u'-', u'unit\xe3\x83\xe2\xa9', u'p10', u'28-300', u'mm', u'pour', u'appareil', u'photo', u'gxr'], label=1)
Row(words=[u'logistic,regression,models,are,neat'], label=2)
